In [4]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import time

In [6]:
df = pd.read_csv('test.csv')
df = df.dropna(subset=['Open', 'Close'])
# Extract Open and Close columns
opens = df['Open'].values.reshape(-1, 1)
closes = df['Close'].values.reshape(-1, 1)

In [8]:
# Step 4: Normalize Data
scaler_open = MinMaxScaler()
scaler_close = MinMaxScaler()
normalized_opens = scaler_open.fit_transform(opens)
normalized_closes = scaler_close.fit_transform(closes)


In [10]:
# Step 5: Prepare Training Data
X_open = normalized_opens[:-1]
y_open = normalized_opens[1:]

X_close = normalized_opens
y_close = normalized_closes

In [19]:
# Open price prediction model
open_model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,)),  # 👈 Correct way to specify input shape
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])
open_model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='mse')

# Close price prediction model
close_model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,)),  # 👈 Correct way to specify input shape
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])
close_model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='mse')

# Train both models
start_time = time.time()
open_model.fit(X_open, y_open, epochs=50, batch_size=1, verbose=0)
close_model.fit(X_close, y_close, epochs=50, batch_size=1, verbose=0)


In [21]:
# Step 7: Predict Next Day Prices
last_open = normalized_opens[-1].reshape(1, 1)
predicted_open_norm = open_model.predict(last_open)[0][0]
predicted_close_norm = close_model.predict(last_open)[0][0]

predicted_open = scaler_open.inverse_transform([[predicted_open_norm]])[0][0]
predicted_close = scaler_close.inverse_transform([[predicted_close_norm]])[0][0]

end_time = time.time()
time_taken = end_time - start_time

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


In [23]:
# Step 8: Output Results
print(f"Number of entries used for training: {len(opens)}")
print(f"Prediction: Next day's opening price is {predicted_open:.2f}, and closing price is {predicted_close:.2f}.")
print(f"Time taken for prediction: {time_taken:.2f} seconds.")

Number of entries used for training: 256
Prediction: Next day's opening price is 23695.09, and closing price is 23607.90.
Time taken for prediction: 65.28 seconds.
